In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd

# Path to your CSV file in Google Drive
csv_file_path = '/content/drive/My Drive/call_data.csv'


# Read the CSV into a pandas DataFrame
df = pd.read_csv(csv_file_path)
# Display the DataFrame
print(df.head())


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-1-983a8c7b291f>:11: DtypeWarning: Columns (1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file_path)


              datetime          expiry_date right  strike_price    open  \
0  2023-03-01 09:16:00  2023-03-29 15:30:00  Call       40100.0  955.00   
1  2023-03-01 09:17:00  2023-03-29 15:30:00  Call       40100.0  977.75   
2  2023-03-01 09:18:00  2023-03-29 15:30:00  Call       40100.0  973.45   
3  2023-03-01 09:20:00  2023-03-29 15:30:00  Call       40100.0  978.60   
4  2023-03-01 09:21:00  2023-03-29 15:30:00  Call       40100.0  994.50   

      high     low  close_option  close_spot  time_to_expiry  log_return  \
0   964.35  951.10        964.35    40411.45        0.077424         NaN   
1   977.75  975.00        975.00    40437.80        0.077422    0.000652   
2   973.45  966.55        966.55    40427.35        0.077420   -0.000258   
3   991.35  978.60        987.65    40459.60        0.077416    0.000797   
4  1015.00  994.50       1010.85    40493.05        0.077414    0.000826   

   sigma_20  
0       NaN  
1       NaN  
2       NaN  
3       NaN  
4       NaN  


In [ ]:
!pip3 install keras
!pip3 install tensorflow
!pip3 install sklearn

from keras.models import Sequential, Model
from keras.layers import Dense, Activation, LeakyReLU, BatchNormalization, LSTM, Input, Concatenate
from keras import backend as K

from keras.callbacks import TensorBoard
from keras.optimizers import Adam
from keras.utils import plot_model
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split


  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
layers = 4
n_timesteps = 20
features = 4
n_batch = 4096
n_epochs = 100

In [ ]:
def make_model():
    close_history = Input((n_timesteps, 1))
    input2 = Input((features,))

    lstm = Sequential()
    lstm.add(LSTM(units=4, input_shape=(n_timesteps, 1), return_sequences=True))
    lstm.add(LSTM(units=4, return_sequences=True))
    lstm.add(LSTM(units=4, return_sequences=True))
    lstm.add(LSTM(units=4, return_sequences=False))
    input1 = lstm(close_history)

    connect = Concatenate()([input1, input2])

    for _ in range(layers - 1):
        connect = Dense(100)(connect)
        connect = BatchNormalization()(connect)
        connect = LeakyReLU()(connect)

    predict = Dense(1, activation='relu')(connect)

    return Model(inputs=[close_history, input2], outputs=predict)

In [ ]:
model = make_model()
model.compile(loss='mse', optimizer=Adam())

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
model.summary()

Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 20, 1)          │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ sequential (Sequential)   │ (None, 4)              │            528 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_1             │ (None, 4)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate (Concatenate) │ (None, 8)              │              0 │ sequential[0][0],      │
│                           │                        │                │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 100)            │            900 │ concatenate[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization       │ (None, 100)            │            400 │ dense[0][0]            │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ leaky_re_lu (LeakyReLU)   │ (None, 100)            │              0 │ batch_normalization[0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 100)            │         10,100 │ leaky_re_lu[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_1     │ (None, 100)            │            400 │ dense_1[0][0]          │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ leaky_re_lu_1 (LeakyReLU) │ (None, 100)            │              0 │ batch_normalization_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_2 (Dense)           │ (None, 100)            │         10,100 │ leaky_re_lu_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_2     │ (None, 100)            │            400 │ dense_2[0][0]          │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ leaky_re_lu_2 (LeakyReLU) │ (None, 100)            │              0 │ batch_normalization_2… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_3 (Dense)           │ (None, 1)              │            101 │ leaky_re_lu_2[0][0]    │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 22,929 (89.57 KB)

 Trainable params: 22,329 (87.22 KB)

 Non-trainable params: 600 (2.34 KB)

In [ ]:
call_df = df
call_df = call_df.dropna(axis=0)
call_df = call_df.drop(columns=['expiry_date', 'right', 'log_return', 'sigma_20'])
# call_df.strike_price = call_df.strike_price / 100
# call_df=call_df.drop(['datetime'],axis=1)
# call_df = df[df.cp_flag == 'C'].drop(['cp_flag'], axis=1)
# put_df = df[df.cp_flag == 'P'].drop(['cp_flag'], axis=1)
call_df['risk_free_rate']=4
call_df.head()




,datetime,strike_price,open,high,low,close_option,close_spot,time_to_expiry,risk_free_rate
20,2023-03-01 09:38:00,40100.0,1030.15,1030.15,1030.15,1030.15,40519.35,0.077382,4
21,2023-03-01 09:39:00,40100.0,1027.55,1027.55,1027.55,1027.55,40523.15,0.077380,4
22,2023-03-01 09:40:00,40100.0,1034.35,1034.35,1023.60,1023.60,40525.65,0.077378,4
23,2023-03-01 09:41:00,40100.0,1020.80,1020.80,1010.00,1010.00,40485.15,0.077376,4
24,2023-03-01 09:42:00,40100.0,1012.50,1018.15,1012.50,1018.15,40489.70,0.077374,4


In [ ]:
padded = np.insert(call_df.close_spot.values, 0, np.array([np.nan] * 20))
call_df=call_df.drop(['close_spot'],axis=1)

In [ ]:
rolled = np.column_stack([np.roll(padded, i) for i in range(20)])
rolled = rolled[~np.isnan(rolled).any(axis=1)]


In [ ]:
rolled.shape


(2163741, 20)

In [ ]:
rolled = np.column_stack((call_df.datetime.values[19:], rolled))

In [ ]:
price_history = pd.DataFrame(data=rolled)

In [ ]:
import pandas as pd
import numpy as np


# Calculate how many NaN rows to add at the start
pad_length = len(call_df) - len(price_history)
price_history_padded = pd.concat([pd.DataFrame(np.nan, index=range(pad_length), columns=price_history.columns), price_history]).reset_index(drop=True)

# Concatenate or join the two DataFrames along columns
joined= pd.concat([call_df, price_history_padded], axis=1)



         strike_price     open     high      low  close_option  \
20            40100.0  1030.15  1030.15  1030.15       1030.15   
21            40100.0  1027.55  1027.55  1027.55       1027.55   
22            40100.0  1034.35  1034.35  1023.60       1023.60   
23            40100.0  1020.80  1020.80  1010.00       1010.00   
24            40100.0  1012.50  1018.15  1012.50       1018.15   
...               ...      ...      ...      ...           ...   
1449240           NaN      NaN      NaN      NaN           NaN   
1449241           NaN      NaN      NaN      NaN           NaN   
1449242           NaN      NaN      NaN      NaN           NaN   
1449243           NaN      NaN      NaN      NaN           NaN   
1449244           NaN      NaN      NaN      NaN           NaN   

         time_to_expiry  risk_free_rate                    0         1  \
20             0.077382             4.0  2023-03-01 10:08:00  40554.95   
21             0.077380             4.0  2023-03-01 10:10:0

In [ ]:
call_df=joined.drop([0],axis=1)

In [ ]:
call_df

,strike_price,open,high,low,close_option,time_to_expiry,risk_free_rate,1,2,3,...,11,12,13,14,15,16,17,18,19,20
20,40100.0,1030.15,1030.15,1030.15,1030.15,0.077382,4.0,40554.95,40540.25,40536.15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21,40100.0,1027.55,1027.55,1027.55,1027.55,0.077380,4.0,40551.3,40554.95,40540.25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,40100.0,1034.35,1034.35,1023.60,1023.60,0.077378,4.0,40569.55,40551.3,40554.95,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23,40100.0,1020.80,1020.80,1010.00,1010.00,0.077376,4.0,40587.8,40569.55,40551.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24,40100.0,1012.50,1018.15,1012.50,1018.15,0.077374,4.0,40593.45,40587.8,40569.55,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1449240,NaN,NaN,NaN,NaN,NaN,NaN,NaN,43914.5,43914.5,43911.5,...,44375.8,44375.8,44395.95,44395.95,44371.7,44371.7,44339.7,44339.7,44330.8,44330.8
1449241,NaN,NaN,NaN,NaN,NaN,NaN,NaN,43913.05,43914.5,43914.5,...,44386.35,44375.8,44375.8,44395.95,44395.95,44371.7,44371.7,44339.7,44339.7,44330.8
1449242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,43913.05,43913.05,43914.5,...,44386.35,44386.35,44375.8,44375.8,44395.95,44395.95,44371.7,44371.7,44339.7,44339.7
1449243,NaN,NaN,NaN,NaN,NaN,NaN,NaN,43913.35,43913.05,43913.05,...,44382.2,44386.35,44386.35,44375.8,44375.8,44395.95,44395.95,44371.7,44371.7,44339.7


In [ ]:
call_X_train, call_X_test, call_y_train, call_y_test = train_test_split(call_df.drop(['open', 'high','low','close_option'], axis=1).values,
                                                                        ((call_df.close_option)).values,
                                                                        test_size=0.01, random_state=42)
# put_X_train, put_X_test, put_y_train, put_y_test = train_test_split(put_df.drop(['best_bid', 'best_offer'], axis=1).values,
#                                                                     ((put_df.best_bid + put_df.best_offer) / 2).values,
#                                                                     test_size=0.01, random_state=42)

In [ ]:
call_X_train=call_X_train.astype(np.float32)
call_X_test=call_X_test.astype(np.float32)
call_y_train=call_y_train.astype(np.float32)
call_y_test=call_y_test.astype(np.float32)

In [ ]:
call_X_train = [call_X_train[:, -20:].reshape(call_X_train.shape[0], 20, 1), call_X_train[:, :4]]
call_X_test = [call_X_test[:, -20:].reshape(call_X_test.shape[0], 20, 1), call_X_test[:, :4]]

In [ ]:
history = model.fit(call_X_train, call_y_train,
                    batch_size=n_batch, epochs=n_epochs,
                    validation_split = 0.01,
                    callbacks=[TensorBoard()],
                    verbose=1)

Epoch 1/100
554/554 ━━━━━━━━━━━━━━━━━━━━ 106s 176ms/step - loss: nan - val_loss: nan
Epoch 2/100
554/554 ━━━━━━━━━━━━━━━━━━━━ 99s 179ms/step - loss: nan - val_loss: nan
Epoch 3/100
554/554 ━━━━━━━━━━━━━━━━━━━━ 141s 177ms/step - loss: nan - val_loss: nan
Epoch 4/100
554/554 ━━━━━━━━━━━━━━━━━━━━ 99s 179ms/step - loss: nan - val_loss: nan
Epoch 5/100
554/554 ━━━━━━━━━━━━━━━━━━━━ 139s 173ms/step - loss: nan - val_loss: nan
Epoch 6/100
554/554 ━━━━━━━━━━━━━━━━━━━━ 141s 173ms/step - loss: nan - val_loss: nan
Epoch 7/100
554/554 ━━━━━━━━━━━━━━━━━━━━ 140s 170ms/step - loss: nan - val_loss: nan
Epoch 8/100
554/554 ━━━━━━━━━━━━━━━━━━━━ 143s 173ms/step - loss: nan - val_loss: nan
Epoch 9/100
554/554 ━━━━━━━━━━━━━━━━━━━━ 142s 172ms/step - loss: nan - val_loss: nan
Epoch 10/100
554/554 ━━━━━━━━━━━━━━━━━━━━ 137s 163ms/step - loss: nan - val_loss: nan
Epoch 11/100
554/554 ━━━━━━━━━━━━━━━━━━━━ 90s 162ms/step - loss: nan - val_loss: nan
Epoch 12/100
554/554 ━━━━━━━━━━━━━━━━━━━━ 143s 163ms/step - loss: 

In [ ]:
# Evaluate the model on test data
test_accuracy = model.evaluate(call_X_test, call_y_test, verbose=1)

# Print the results

print(f"Test Accuracy: {test_accuracy}")

In [ ]:
# Evaluate the model on the test data
test_loss, test_metrics = model.evaluate(call_X_test, call_y_test, verbose=1)

print(f"Test Loss: {test_loss}")
print(f"Test Metrics: {test_metrics}")


In [ ]:
# Making predictions on the test data
predictions = model.predict(call_X_test)

# If you want to convert the predicted probabilities to class labels:
predicted_classes = predictions.argmax(axis=-1)  # assuming you're doing classification
print(predicted_classes)